In [ ]:
# uncomment and run if mismo is not installed
# %pip install -q git+https://github.com/NickCrews/mismo@main

In [1]:
from __future__ import annotations

import ibis


ibis.options.interactive = True


Many real-world datasets contain errors due to causes such as manual data entry, incorrect data processing and inconsistent formatting. Therefore it's often useful to make use of string similarity measures that can quantify how close two strings are to each other by accounting for common types of string manipulations. These are defined to give a score between 0 and 1 which indicate minimal and maximal similarity respectively. mismo currently implements the following string similarity measures that are suitable for different use-cases:

- `Jaro` - a measure of the similarity between two strings given the number of matching characters and transpositions and their length.
- `Jaro-Winkler` - a modification of the Jaro similarity that uses a prefix scale to give more favourable weightings to strings that match at the start.
- `Jaccard` - a measure of the number of overlapping sets of words in two strings.

In addition, the following edit distance measures are defined along with equivalent similarities that are normalized using string lengths.
- `Levenshtein` - a measure of the distance between two strings based on the number of deletions, insertions and substitutions.
- `Damerau-Levenshtein` - an extension of `Levenshtein` that includes transpositions.

Let's explore how these work in practice using the patents dataset. We will generate pairs by blocking on the `label_true` column

In [2]:
from mismo.playdata import load_patents
patents = load_patents()
patents

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │     0.00 │  0.000000 │ GABRIEL MARINUS  MEESTERS**RUDOLF CAROLUS  BARENDSE**ARIE KARST  KIES**ALEXANDE… │ C12N**A61K**A23L**A23J**A23K**A01H**B01J**C12R**C07D**A61P**B01D │
│         … │          … │ …                    │ …                            │        … │         … │ …                                                                                │ …  

In [3]:
from mismo.block import KeyBlocker
blocked = KeyBlocker("label_true")(patents, patents)

A comparison table of these string similarity measures can be generated using `mismo.eda.string_comparator_scores`

In [4]:
from mismo.eda import string_comparator_score_chart, string_comparator_scores
scores = string_comparator_scores(blocked.limit(20), "name_l", "name_r")

In [5]:
scores

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ string1                         ┃ string2                        ┃ jaro_similarity ┃ jaro_winkler_similarity ┃ jaccard_similarity ┃ levenshtein_ratio ┃ damerau_levenshtein_ratio ┃ levenshtein_distance ┃ damerau_levenshtein_distance ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                          │ string                         │ float64         │ float64                 │ float64            │ float64           │ float64                   │ int64                │ int64                        │
├─────────────────────────────────┼────────────────────────────────┼─────────────────┼─────────────────────────┼────────────────────┼───────────────────┼───────────────────────────┼──────────────────────┼──────────────────────────────┤
│ * AKZO NOBEL N.V.               │ * AKZO NOBEL NV                │        0.960784 │                0.976471 │           0.916667 │          0.882353 │                  0.882353 │                    2 │                            2 │
│ * ALCATEL N.V.                  │ * ALCATEL N.V.                 │        1.000000 │                1.000000 │           1.000000 │          1.000000 │                  1.000000 │                    0 │                            0 │
│ * ALCATEL N.V.                  │ * ALCATEL N.V.                 │        1.000000 │                1.000000 │           1.000000 │          1.000000 │                  1.000000 │                    0 │                            0 │
│ * CANON EUROPA N.V              │ * CANON EUROPA N.V.            │        0.982456 │                0.989474 │           1.000000 │          0.947368 │                  0.947368 │                    1 │                            1 │
│ * CANON EUROPA N.V              │ * CANON EUROPA NV              │        0.981481 │                0.988889 │           0.916667 │          0.944444 │                  0.944444 │                    1 │                            1 │
│ * DSM N.V.                      │ DSM N.V.                       │        0.850000 │                0.850000 │           0.875000 │          0.800000 │                  0.800000 │                    2 │                            2 │
│ * HUNTER DOUGLAS INDUSTRIES B V │ * HUNTER DOUGLAS INDUSTRIES BV │        0.989247 │                0.993548 │           1.000000 │          0.967742 │                  0.967742 │                    1 │                            1 │
│ * HUNTER DOUGLAS INDUSTRIES B V │ * HUNTER DOUGLAS INDUSTRIES BV │        0.989247 │                0.993548 │           1.000000 │          0.967742 │                  0.967742 │                    1 │                            1 │
│ * HUNTER DOUGLAS INDUSTRIES BV  │ * HUNTER DOUGLAS INDUSTRIES BV │        1.000000 │                1.000000 │           1.000000 │          1.000000 │                  1.000000 │                    0 │                            0 │
│ * HUNTER DOUGLAS INDUSTRIES BV  │ * HUNTER DOUGLAS INDUSTRIES BV │        1.000000 │                1.000000 │           1.000000 │          1.000000 │                  1.000000 │                    0 │                            0 │
│ …                               │ …                              │               … │                       … │                  … │                 … │                         … │                    … │                            … │
└─────────────────────────────────┴────────────────────────────────┴─────────────────┴─────────────────────────┴────────────────────┴───────────────────┴───────────────────────────┴──────────────────────┴────────────────────

These can be visually represented using `mismo.eda.string_comparator_score_chart` which plots a heatmap of the similarity and distance measures.

In [6]:
chart = string_comparator_score_chart(blocked.limit(20), "name_l", "name_r")
chart

alt.HConcatChart(...)